# Fine Tuning with MobileNetV2 Backbone

# Imports






In [ ]:
# !pip uninstall tf-keras
# !pip install tensorflow==2.16.1

In [ ]:
import keras
import tensorflow as tf
print("Keras Current Version:", keras.__version__, "Tensorflow Current Version:", tf.__version__)

In [ ]:
import os, random, datetime
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.metrics import Accuracy, AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models  import load_model

# Tensorflow ve Keras kütüphaneleri
import tensorflow as tf
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as preprocess_input_mobilenetv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Conv2D, Flatten, MaxPooling2D, Dropout, SpatialDropout2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


# Functions

In [ ]:
def get_image_paths(root_dir, num_images=None):
    all_images = []
    for extension in ['*.jpg', '*.jpeg', '*.png']:
        all_images.extend(glob(os.path.join(root_dir, '**', extension), recursive=True))
    if num_images is None:
        return all_images
    else:
        return random.sample(all_images, min(num_images, len(all_images)))

def display_images(img_list):
    plt.figure(figsize=(15, 6))
    for i, img_path in enumerate(img_list):
        img = image.load_img(img_path)
        img = image.img_to_array(img, dtype=np.uint8)
        plt.subplot(2, 5, i + 1)
        plt.imshow(img.squeeze())
        plt.axis('off')
        plt.title(f'Image {i+1}')
    plt.tight_layout()
    plt.show()


def print_predicted_classes(predicted_classes):
    for full_path, (label, probability) in predicted_classes.items():
        filename = os.path.basename(full_path)
        print(f"{filename}: {label} ({probability:.2f}%)")

def plot_training_history(history, train_loss='loss', train_metric='accuracy', val_loss='val_loss', val_metric='val_accuracy'):

    #Loss
    plt.figure(figsize=(10, 5))
    plt.plot(history.history[train_loss], label='Training Loss')
    plt.plot(history.history[val_loss], label='Validation Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Metrics
    plt.figure(figsize=(10, 5))
    plt.plot(history.history[train_metric], label=f"Training: {train_metric}")
    plt.plot(history.history[val_metric], label=f"Validation: {val_metric}")
    plt.title(f'Training and Validation {train_metric} Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel(f'train_metric')
    plt.legend()
    plt.show()


# Data Preparation & Augmentation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/Garbage classification'

In [ ]:
img_list = get_image_paths(dir_path)

len(img_list)

In [ ]:
# 501 glass
# 594 paper
# 403 cardboard
# 482 plastic
# 410 metal
# 137 trash

In [ ]:
display_images(img_list[0:10])

In [ ]:
train = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)


val = ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)


In [ ]:
train_generator=train.flow_from_directory(dir_path,
                                          target_size=(224, 224),
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset='training')

validation_generator=val.flow_from_directory(dir_path,
                                        target_size=(224, 224),
                                        batch_size=251,
                                        class_mode='categorical',
                                        subset='validation')


# Frozen Layers


## MobilenetV2 Backbone

In [ ]:
mobilenet_backbone = MobileNetV2(weights='imagenet',

                                 include_top=False,

                                 input_shape=(224, 224, 3))

#alpha=1.0
#classes

In [ ]:
mobilenet_backbone.summary()

In [ ]:
print(f"Total number of layers in MobileNetV2: {len(mobilenet_backbone.layers)}")

In [ ]:
block_count = 0
for layer in mobilenet_backbone.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) and 'expand' in layer.name:
        block_count += 1

print(f"Number of logical 'blocks' in MobileNetV2: {block_count}")

In [ ]:
for layer in mobilenet_backbone.layers:
    print(layer.name, layer.trainable)

In [ ]:
for layer in mobilenet_backbone.layers[-11:]:
    print(layer.name, layer.trainable)

In [ ]:
for layer in mobilenet_backbone.layers:
    layer.trainable = False

In [ ]:
for layer in mobilenet_backbone.layers:
    print(layer.name, layer.trainable)

In [ ]:
for layer in mobilenet_backbone.layers[-11:]:
    layer.trainable = True

In [ ]:
for layer in mobilenet_backbone.layers:
    print(layer.name, layer.trainable)

## Training

In [ ]:
x = GlobalAveragePooling2D()(mobilenet_backbone.output)

In [ ]:
x = Dense(6, activation='softmax')(x)

In [ ]:
fine_tuning_model = Model(inputs=mobilenet_backbone.input, outputs=x)

In [ ]:
metrics = ["accuracy", AUC(name='auc', multi_label=True)]

In [ ]:
from tensorflow.keras.optimizers import SGD

In [ ]:
optimizer=SGD(learning_rate=0.0001, momentum=0.9, nesterov=True)

In [ ]:
fine_tuning_model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=metrics)

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               restore_best_weights=True,
                               verbose=1)


model_checkpoint = ModelCheckpoint('mobilenetv2_finetuned.keras',
                                   monitor='val_loss',
                                   save_best_only=True,
                                   save_weights_only=False,
                                   verbose=1)

In [ ]:
start_time = datetime.datetime.now()

fine_tuning_model_history = fine_tuning_model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint]
)

end_time = datetime.datetime.now()

total_duration = end_time - start_time
print("Training Time:", total_duration)

In [ ]:
plot_training_history(fine_tuning_model_history, train_loss='loss', train_metric='accuracy', val_loss='val_loss', val_metric='val_accuracy')

In [ ]:
plot_training_history(fine_tuning_model_history, train_loss='loss', train_metric='auc', val_loss='val_loss', val_metric='val_auc')

In [ ]:
val_loss, val_accuracy, val_auc = fine_tuning_model.evaluate(validation_generator, verbose=0)
print(f"Loss: {val_loss}")
print(f"Accuracy: {val_accuracy}")
print(f"AUC: {val_auc}")

## Prediction & Inference

In [ ]:
waste_labels = {0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

In [ ]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/Garbage classification'

In [ ]:
img_list = get_image_paths(dir_path, 10)

In [ ]:
garbage_tuned_model = load_model('/content/mobilenetv2_finetuned.keras')

In [ ]:
def preprocess_mobilenet(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input_mobilenetv2(img_array)
    return img_array

def predict_mobilenet(model, img_array, class_labels):
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = class_labels[predicted_class_idx]
    probability = np.max(predictions[0])
    return predicted_class, probability

def visualise_preds_mobilenet(model, image_paths, class_labels, visualize=False):
    results = {}
    for img_path in image_paths:
        img_array = preprocess_mobilenet(img_path)
        label, probability = predict_mobilenet(model, img_array, class_labels)
        results[img_path] = (label, probability)
        if visualize:
            plt.figure(figsize=(5, 5))
            plt.imshow(image.load_img(img_path))
            plt.title(f"Predicted: {label} ({probability:.2f}%)")
            plt.axis('off')
            plt.show()

    return results



In [ ]:
predicted_classes = visualise_preds_mobilenet(garbage_tuned_model, img_list, waste_labels, True)

In [ ]:
print_predicted_classes(predicted_classes)

In [ ]:
img_list2 = get_image_paths(dir_path, 50)

In [ ]:
predicted_classes2 = visualise_preds_mobilenet(garbage_tuned_model, img_list2, waste_labels, False)

In [ ]:
print_predicted_classes(predicted_classes2)

# Full Network Fine Tuning



## Data Preparation & Augmentation

In [ ]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/Garbage classification'

In [ ]:
img_list = get_image_paths(dir_path)

len(img_list)

In [ ]:
train = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)


val = ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)

train_generator=train.flow_from_directory(dir_path,
                                          target_size=(224, 224),
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset='training')

validation_generator=val.flow_from_directory(dir_path,
                                        target_size=(224, 224),
                                        batch_size=251,
                                        class_mode='categorical',
                                        subset='validation')


## Model

In [ ]:
mobilenet_backbone_full = MobileNetV2(weights='imagenet',
                                 include_top=False,
                                 input_shape=(224, 224, 3))

In [ ]:
for layer in mobilenet_backbone_full.layers:
    layer.trainable = True

In [ ]:
x = GlobalAveragePooling2D()(mobilenet_backbone_full.output)

x = Dense(6, activation='softmax')(x)

In [ ]:
full_network_fine_tuning = Model(inputs=mobilenet_backbone_full.input, outputs=x)

In [ ]:
metrics = ["accuracy", AUC(name='auc', multi_label=True)]

In [ ]:
from tensorflow.keras.optimizers import SGD

In [ ]:
optimizer=SGD(learning_rate=0.0001, momentum=0.9, nesterov=True)

In [ ]:
full_network_fine_tuning.compile(optimizer,
              loss='categorical_crossentropy',
              metrics=metrics)

In [ ]:
early_stopping_full = EarlyStopping(monitor='val_loss',
                               patience=10,
                               restore_best_weights=True,
                               verbose=1)


model_checkpoint_full = ModelCheckpoint('mobilenetv2_full_network.keras',
                                   monitor='val_loss',
                                   save_best_only=True,
                                   save_weights_only=False,
                                   verbose=1)


## Training

In [ ]:
start_time = datetime.datetime.now()

full_network_history = full_network_fine_tuning.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[early_stopping_full, model_checkpoint_full]
)

end_time = datetime.datetime.now()

total_duration = end_time - start_time
print("Training Time:", total_duration)

# Model Performance

In [ ]:
plot_training_history(full_network_history, train_loss='loss', train_metric='accuracy', val_loss='val_loss', val_metric='val_accuracy')

In [ ]:
plot_training_history(full_network_history, train_loss='loss', train_metric='auc', val_loss='val_loss', val_metric='val_auc')

In [ ]:
val_loss, val_accuracy, val_auc = full_network_fine_tuning.evaluate(validation_generator, verbose=0)
print(f"Loss: {val_loss}")
print(f"Accuracy: {val_accuracy}")
print(f"AUC: {val_auc}")

## Prediction

In [ ]:
waste_labels = {0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

In [ ]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/Garbage classification'

In [ ]:
img_list = get_image_paths(dir_path, 10)

In [ ]:
garbage_full_tuned_model = load_model('mobilenetv2_full_network.keras')

In [ ]:
def preprocess_mobilenet(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input_mobilenetv2(img_array)
    return img_array

def predict_mobilenet(model, img_array, class_labels):
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = class_labels[predicted_class_idx]
    probability = np.max(predictions[0])
    return predicted_class, probability

def visualise_preds_mobilenet(model, image_paths, class_labels, visualize=False):
    results = {}
    for img_path in image_paths:
        img_array = preprocess_mobilenet(img_path)
        label, probability = predict_mobilenet(model, img_array, class_labels)
        results[img_path] = (label, probability)
        if visualize:
            plt.figure(figsize=(5, 5))
            plt.imshow(image.load_img(img_path))
            plt.title(f"Predicted: {label} ({probability:.2f}%)")
            plt.axis('off')
            plt.show()

    return results

In [ ]:
predicted_classes = visualise_preds_mobilenet(garbage_full_tuned_model, img_list, waste_labels, True)

In [ ]:
print_predicted_classes(predicted_classes)